In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [3]:
spark = SparkSession.builder \
    .appName("KPI_Job") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.cores", "2") \
    .getOrCreate()

In [4]:
spark

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType
order_items_df = spark.read.option("header", True) \
    .option("inferSchema", True) \
    .csv("../Data/order_items/*.csv")
order_items_df = order_items_df.limit(1000)
order_items_df.cache()

In [ ]:
orders_df = spark.read.option("header", True).csv("../Data/orders/*.csv", inferSchema=True)

In [ ]:
products_df = spark.read.option("header", True).csv("../Data/products.csv", inferSchema=True)